<H1>Segmenting and Clustering Neighborhoods in Toronto</H1>
<H2>Coursera Capstone Project Data Science</H2>

This notebook consists of multiple sections. 

First import and install the necessary libraries.

In [2]:
!conda install -c conda-forge geopy --yes

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2019.9.11  |       hecc5488_0         144 KB  conda-forge
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    certifi-2019.9.11          |           py36_0         147 KB  conda-forge
    openssl-1.1.1c             |       h516909a_0         2.1 MB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.49-py_0         conda-forge
    geopy:           1.20.0-py_0       conda-forge

The following packages will be UPDATED:

    ca-

In [3]:
!conda install -c conda-forge folium --yes

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    altair-3.2.0               |           py36_0         770 KB  conda-forge
    folium-0.10.0              |             py_0          59 KB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         882 KB

The following NEW packages will be INSTALLED:

    altair:  3.2.0-py36_0 conda-forge
    branca:  0.3.1-py_0   conda-forge
    folium:  0.10.0-py_0  conda-forge
    vincent: 0.4.4-py_1   conda-forge


vincent-0.4.4        | 28 KB     | ##################################### |

In [4]:
!conda install -c conda-forge beautifulsoup4 --yes

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - beautifulsoup4


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    beautifulsoup4-4.8.0       |           py36_0         144 KB  conda-forge

The following packages will be UPDATED:

    beautifulsoup4: 4.7.1-py36_1 --> 4.8.0-py36_0 conda-forge


beautifulsoup4-4.8.0 | 144 KB    | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done


In [5]:
!conda install -c conda-forge geocoder --yes

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geocoder


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ratelim-0.1.6              |             py_2           6 KB  conda-forge
    geocoder-1.38.1            |             py_1          53 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          59 KB

The following NEW packages will be INSTALLED:

    geocoder: 1.38.1-py_1 conda-forge
    ratelim:  0.1.6-py_2  conda-forge


ratelim-0.1.6        | 6 KB      | ##################################### | 100% 
geocoder-1.38.1      | 53 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done


In [6]:
# first import some libraries necessary for the actions below
import requests
import lxml.html as lh
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup


<H2>Table of contents</H2>

<font size = 3>

1. <a href="#item1">Scrape table from Wikipedia</a>

2. <a href="#item2">Create a data frame with FourSquare data</a>

3. <a href="#item3">Explore and cluster the neighborhoods in Toronto</a>
</font>
</div>

<H3>Scrape table from Wikipedia</H3>

Scrape the table with the Toronto postal codes from Wikipedia and put them in a pandas dataframe.

In [7]:
# get the page from Wikipedia
scrape_url=requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
# use BeautifulSoup to import the table
soup = BeautifulSoup(scrape_url.content,'lxml')
table = soup.find_all('table')[0]
# and put it in a Pandas dataframe
df = pd.read_html(str(table))[0]

In [13]:
# show first 5 rows of the newly create Data Frame
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [14]:
# check how many instances of "Not Assigned" we have in Borough, and drop those items
df['Borough'].value_counts()

Not assigned        77
Etobicoke           45
North York          38
Scarborough         37
Downtown Toronto    37
Central Toronto     17
West Toronto        13
York                 9
East Toronto         7
East York            6
Mississauga          1
Queen's Park         1
Name: Borough, dtype: int64

In [15]:
# We don't need the 'Not assigned' boroughs
df.drop(df.loc[df['Borough'] == 'Not assigned'].index, inplace=True)
# reset the index since we removed rows.
df.reset_index(drop=True, inplace=True)
df.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


In [16]:
# check if the Not assigned items are indeed removed
df['Borough'].value_counts()

Etobicoke           45
North York          38
Scarborough         37
Downtown Toronto    37
Central Toronto     17
West Toronto        13
York                 9
East Toronto         7
East York            6
Mississauga          1
Queen's Park         1
Name: Borough, dtype: int64

In [17]:
# groupby and apply join to have 1 line per post code
df_joined = df.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(','.join).reset_index()

In [18]:
df_check1 = df_joined.loc[df_joined['Postcode'] == 'M5A']
df_check2 = df_joined.loc[df_joined['Neighbourhood'] == 'Not assigned']
# check that the Neighbourhoods are indeed combined, with the example from Postcode M5A
print(df_check1)
# check which Neighbourhood is not assigned.
print(df_check2)

   Postcode           Borough             Neighbourhood
53      M5A  Downtown Toronto  Harbourfront,Regent Park
   Postcode       Borough Neighbourhood
85      M7A  Queen's Park  Not assigned


In [19]:
# replace Not assigned with the name of the Borough. 
df_joined['Neighbourhood'].replace('Not assigned', 'Queen\'s Park', inplace=True)
#df_check3 = df_joined.loc[df_joined['Neighbourhood'] == 'Not assigned']
#df_check3
#df_joined.head()

This concludes this part of the exercise.

In [20]:
df_joined.shape

(103, 3)

<H3>Create a data frame with FourSquare data</H3>

In this task I'm going to find the latitude and the longitude coordinates of each neighbourhood.

In [21]:
import geocoder # import geocoder

In [79]:
#g = geocoder.google('Mountain View, CA')
#g.latlng

In [22]:
# For some reason the geocoder method doesn't work for me. So I decided to use the provided geospatial data.
# first download and import the csv in a data frame.
filename = "https://cocl.us/Geospatial_data"
df_gsdata = pd.read_csv(filename)
df_gsdata.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [23]:
# rename column Postal Code to Postcode
df_gsdata.rename(columns={'Postal Code':'Postcode'}, inplace=True)
df_gsdata.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [24]:
# merge the two data frames together
df_new = pd.merge(df_joined, df_gsdata, on='Postcode')
df_new.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


<H3>Explore and cluster the neighborhoods in Toronto</H3>

Third part of the assignment:

Explore and cluster the neighborhoods in Toronto. You can decide to work with only boroughs that contain the word Toronto and then replicate the same analysis we did to the New York City data. It is up to you.

Just make sure:

to add enough Markdown cells to explain what you decided to do and to report any observations you make.
to generate maps to visualize your neighborhoods and how they cluster together.

In [25]:
import numpy as np
import json

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print('Libraries imported.')

Libraries imported.


With the libraries imported, let's make a map of Toronto, Canada.
First create a map with the focus on Toronto, then plot the neighborhoods from our previous tasks on the map, using the dataframe df_new.

In [26]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="ca_explorer")
location = geolocator.geocode(address)
to_latitude = location.latitude
to_longitude = location.longitude
print('The geograpical coordinates of Toronto are {}, {}.'.format(to_latitude, to_longitude))

The geograpical coordinates of Toronto are 43.653963, -79.387207.


<b>Plot the locations of the neighborhoods on the Toronto map</b>

In [27]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[to_latitude, to_longitude], zoom_start=10)

neighborhoods = df_new
# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Now let's zoom in to Down Town Toronto. First modify our data frame.

In [28]:
downtown = df_new.loc[df_new['Postcode'] == 'M5A']
# Extract the latitude and the longitude from the data frame
latitude_dt = downtown.iloc[0, 3]
longitude_dt = downtown.iloc[0, 4]

<b>Now use the Foursquare API</b>

In [29]:
# define the Foursquare api
CLIENT_ID = 'WVPNGH1SOPRB2TGFZISLMACZQKWIYYLBEAY2DMTLYK2IH2I2' # your Foursquare ID
CLIENT_SECRET = '5LWU1FW00NVCZB4QH0JJ4KTHJ0LUFMPTAY1HULJWU0MPOLRG' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

<b>Let's see the various categories of venues in downtown Toronto.</b>

In [30]:
search_query = ''
radius = 100
latitude = latitude_dt
longitude = longitude_dt
LIMIT = 100

In [31]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
#url

In [32]:
results = requests.get(url).json()
#results

In [33]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '4bf58dd8d48988d163941735', 'name': 'P...",False,4bc70f5d14d7952126a066e9,420 king st E,CA,Toronto,Canada,NaN,75,"[420 king st E, Toronto ON, Canada]","[{'label': 'display', 'lat': 43.65465604258614...",43.654656,-79.359871,NaN,NaN,ON,Sackville Playground,v-1568985265,NaN
1,"[{'id': '4bf58dd8d48988d124951735', 'name': 'A...",False,4dc9d4d9d16495ca5add0803,475 King Street East,CA,Toronto,Canada,Sackville Street,10,"[475 King Street East (Sackville Street), Toro...","[{'label': 'display', 'lat': 43.65419500779484...",43.654195,-79.360545,NaN,M4M 2T7,ON,Cam's Auto Service,v-1568985265,NaN
2,"[{'id': '4bf58dd8d48988d143941735', 'name': 'B...",False,4ae5b91ff964a520a6a121e3,457 King St. E,CA,Toronto,Canada,Gilead Place,54,"[457 King St. E (Gilead Place), Toronto ON M5A...","[{'label': 'display', 'lat': 43.65394694263529...",43.653947,-79.361149,NaN,M5A 1L6,ON,Morning Glory Cafe,v-1568985265,39686393
3,"[{'id': '4bf58dd8d48988d110951735', 'name': 'S...",False,5d266686f3c3c10023171ace,NaN,CA,Toronto,Canada,NaN,32,"[Toronto ON M5A 1L4, Canada]","[{'label': 'display', 'lat': 43.654175, 'lng':...",43.654175,-79.361020,NaN,M5A 1L4,ON,Hastings Barbershop Corktown,v-1568985265,NaN
4,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",False,53b8466a498e83df908c3f21,368 King St E,CA,Toronto,Canada,at Trinity St,122,"[368 King St E (at Trinity St), Toronto ON, Ca...","[{'label': 'display', 'lat': 43.65355870959944...",43.653559,-79.361809,NaN,NaN,ON,Tandem Coffee,v-1568985265,NaN


In [34]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Sackville Playground,Park,420 king st E,CA,Toronto,Canada,NaN,75,"[420 king st E, Toronto ON, Canada]","[{'label': 'display', 'lat': 43.65465604258614...",43.654656,-79.359871,NaN,NaN,ON,4bc70f5d14d7952126a066e9
1,Cam's Auto Service,Automotive Shop,475 King Street East,CA,Toronto,Canada,Sackville Street,10,"[475 King Street East (Sackville Street), Toro...","[{'label': 'display', 'lat': 43.65419500779484...",43.654195,-79.360545,NaN,M4M 2T7,ON,4dc9d4d9d16495ca5add0803
2,Morning Glory Cafe,Breakfast Spot,457 King St. E,CA,Toronto,Canada,Gilead Place,54,"[457 King St. E (Gilead Place), Toronto ON M5A...","[{'label': 'display', 'lat': 43.65394694263529...",43.653947,-79.361149,NaN,M5A 1L6,ON,4ae5b91ff964a520a6a121e3
3,Hastings Barbershop Corktown,Salon / Barbershop,NaN,CA,Toronto,Canada,NaN,32,"[Toronto ON M5A 1L4, Canada]","[{'label': 'display', 'lat': 43.654175, 'lng':...",43.654175,-79.361020,NaN,M5A 1L4,ON,5d266686f3c3c10023171ace
4,Tandem Coffee,Coffee Shop,368 King St E,CA,Toronto,Canada,at Trinity St,122,"[368 King St E (at Trinity St), Toronto ON, Ca...","[{'label': 'display', 'lat': 43.65355870959944...",43.653559,-79.361809,NaN,NaN,ON,53b8466a498e83df908c3f21
5,Advantage Car and Truck Rentals,Rental Car Location,431 Queen St E,CA,Toronto,Canada,NaN,209,"[431 Queen St E, Toronto ON M5A 1T5, Canada]","[{'label': 'display', 'lat': 43.6561374, 'lng'...",43.656137,-79.360721,NaN,M5A 1T5,ON,58d423c99398ab4da84d0a7a
6,Oldtown Bodega,Café,NaN,CA,Toronto,Canada,NaN,34,"[Toronto ON M5A 1L6, Canada]","[{'label': 'display', 'lat': 43.653966, 'lng':...",43.653966,-79.360752,NaN,M5A 1L6,ON,5bdc6c2bba57b4002c4c71a8
7,392A King St. East,Office,392A King St. East,CA,Toronto,Canada,NaN,179,"[392A King St. East, Toronto ON, Canada]","[{'label': 'display', 'lat': 43.65335432, 'lng...",43.653354,-79.362485,NaN,NaN,ON,4c34a5dd452620a11c6e250f
8,Thruway Muffler / Downtown Brake,Building,475 King St. E,CA,Toronto,Canada,NaN,17,"[475 King St. E, Toronto ON M5A 1L6, Canada]","[{'label': 'display', 'lat': 43.65424167246142...",43.654242,-79.360853,NaN,M5A 1L6,ON,4d067a2b26adb1f702f9dc70
9,Synecticsworld,None,2 Gilead Place,CA,Toronto,Canada,Eastern Ave. & Gilead Pl.,119,"[2 Gilead Place (Eastern Ave. & Gilead Pl.), T...","[{'label': 'display', 'lat': 43.65326, 'lng': ...",43.653260,-79.360095,NaN,NaN,ON,4b745564f964a520dbd52de3


In [35]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13)

for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.Circle(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

In [36]:
dataframe_filtered['categories'].value_counts()

Office                                      13
Automotive Shop                              8
Furniture / Home Store                       4
Park                                         3
Art Gallery                                  3
Rental Car Location                          3
Hardware Store                               2
Design Studio                                2
Flower Shop                                  2
Light Rail Station                           2
Dog Run                                      2
Building                                     2
Church                                       2
Auto Dealership                              2
Residential Building (Apartment / Condo)     2
Assisted Living                              2
Italian Restaurant                           2
Coffee Shop                                  2
Monument / Landmark                          1
Men's Store                                  1
City Hall                                    1
Conference Ro

In [37]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [38]:
toronto_venues = getNearbyVenues(names=df_new['Neighbourhood'],
                                   latitudes=df_new['Latitude'],
                                   longitudes=df_new['Longitude']
                                  )


Rouge,Malvern
Highland Creek,Rouge Hill,Port Union
Guildwood,Morningside,West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park,Ionview,Kennedy Park
Clairlea,Golden Mile,Oakridge
Cliffcrest,Cliffside,Scarborough Village West
Birch Cliff,Cliffside West
Dorset Park,Scarborough Town Centre,Wexford Heights
Maryvale,Wexford
Agincourt
Clarks Corners,Sullivan,Tam O'Shanter
Agincourt North,L'Amoreaux East,Milliken,Steeles East
L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview,Henry Farm,Oriole
Bayview Village
Silver Hills,York Mills
Newtonbrook,Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park,Don Mills South
Bathurst Manor,Downsview North,Wilson Heights
Northwood Park,York University
CFB Toronto,Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Woodbine Gardens,Parkview Hill
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The Danforth West,Riverdale
The Beaches West,Indi

In [39]:
print(toronto_venues.shape)
toronto_venues.head()

(2245, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge,Malvern",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,"Rouge,Malvern",43.806686,-79.194353,Interprovincial Group,43.805630,-79.200378,Print Shop
2,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
3,"Guildwood,Morningside,West Hill",43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place
4,"Guildwood,Morningside,West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store


In [40]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide,King,Richmond",100,100,100,100,100,100
Agincourt,5,5,5,5,5,5
"Agincourt North,L'Amoreaux East,Milliken,Steeles East",2,2,2,2,2,2
"Albion Gardens,Beaumond Heights,Humbergate,Jamestown,Mount Olive,Silverstone,South Steeles,Thistletown",11,11,11,11,11,11
"Alderwood,Long Branch",9,9,9,9,9,9
"Bathurst Manor,Downsview North,Wilson Heights",18,18,18,18,18,18
Bayview Village,4,4,4,4,4,4
"Bedford Park,Lawrence Manor East",21,21,21,21,21,21
Berczy Park,56,56,56,56,56,56


In [41]:
print('There are {} unique categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 275 unique categories.


In [42]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [43]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,"Adelaide,King,Richmond",0.000000,0.0,0.00000,0.0000,0.0000,0.0000,0.000,0.000,0.000,...,0.0,0.020000,0.00000,0.000000,0.000000,0.000000,0.010000,0.0,0.00000,0.0
1,Agincourt,0.000000,0.0,0.00000,0.0000,0.0000,0.0000,0.000,0.000,0.000,...,0.0,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.0,0.00000,0.0
2,"Agincourt North,L'Amoreaux East,Milliken,Steel...",0.000000,0.0,0.00000,0.0000,0.0000,0.0000,0.000,0.000,0.000,...,0.0,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.0,0.00000,0.0
3,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",0.000000,0.0,0.00000,0.0000,0.0000,0.0000,0.000,0.000,0.000,...,0.0,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.0,0.00000,0.0
4,"Alderwood,Long Branch",0.000000,0.0,0.00000,0.0000,0.0000,0.0000,0.000,0.000,0.000,...,0.0,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.0,0.00000,0.0
5,"Bathurst Manor,Downsview North,Wilson Heights",0.000000,0.0,0.00000,0.0000,0.0000,0.0000,0.000,0.000,0.000,...,0.0,0.000000,0.00000,0.055556,0.000000,0.000000,0.000000,0.0,0.00000,0.0
6,Bayview Village,0.000000,0.0,0.00000,0.0000,0.0000,0.0000,0.000,0.000,0.000,...,0.0,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.0,0.00000,0.0
7,"Bedford Park,Lawrence Manor East",0.000000,0.0,0.00000,0.0000,0.0000,0.0000,0.000,0.000,0.000,...,0.0,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.0,0.00000,0.0
8,Berczy Park,0.000000,0.0,0.00000,0.0000,0.0000,0.0000,0.000,0.000,0.000,...,0.0,0.017857,0.00000,0.000000,0.000000,0.000000,0.000000,0.0,0.00000,0.0
9,"Birch Cliff,Cliffside West",0.000000,0.0,0.00000,0.0000,0.0000,0.0000,0.000,0.000,0.000,...,0.0,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.0,0.00000,0.0


In [44]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide,King,Richmond----
         venue  freq
0  Coffee Shop  0.08
1         Café  0.05
2          Bar  0.04
3   Steakhouse  0.04
4   Restaurant  0.03


----Agincourt----
                venue  freq
0      Sandwich Place   0.2
1      Breakfast Spot   0.2
2  Chinese Restaurant   0.2
3              Lounge   0.2
4        Skating Rink   0.2


----Agincourt North,L'Amoreaux East,Milliken,Steeles East----
                      venue  freq
0                Playground   0.5
1                      Park   0.5
2            Medical Center   0.0
3  Mediterranean Restaurant   0.0
4               Men's Store   0.0


----Albion Gardens,Beaumond Heights,Humbergate,Jamestown,Mount Olive,Silverstone,South Steeles,Thistletown----
                 venue  freq
0        Grocery Store  0.18
1  Fried Chicken Joint  0.09
2          Pizza Place  0.09
3             Pharmacy  0.09
4       Sandwich Place  0.09


----Alderwood,Long Branch----
          venue  freq
0   Pizza Place  0.22
1   Coffee Shop  0.11
2 

In [45]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [46]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Coffee Shop,Café,Bar,Steakhouse,Cosmetics Shop,Thai Restaurant,Hotel,Restaurant,Burger Joint,American Restaurant
1,Agincourt,Sandwich Place,Lounge,Breakfast Spot,Skating Rink,Chinese Restaurant,Dumpling Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore
2,"Agincourt North,L'Amoreaux East,Milliken,Steel...",Playground,Park,Eastern European Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Electronics Store
3,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",Grocery Store,Pharmacy,Fried Chicken Joint,Pizza Place,Beer Store,Discount Store,Sandwich Place,Japanese Restaurant,Fast Food Restaurant,Coffee Shop
4,"Alderwood,Long Branch",Pizza Place,Coffee Shop,Skating Rink,Sandwich Place,Pub,Dance Studio,Pharmacy,Gym,Ethiopian Restaurant,Empanada Restaurant


In [47]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 4, 1, 1, 1, 1, 1, 1, 1], dtype=int32)

In [52]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df_new

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

toronto_merged.head() # check the last columns!

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353,1.0,Fast Food Restaurant,Print Shop,Eastern European Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Electronics Store
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497,3.0,Bar,Women's Store,Electronics Store,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Empanada Restaurant,Discount Store
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711,1.0,Pizza Place,Breakfast Spot,Rental Car Location,Intersection,Electronics Store,Mexican Restaurant,Medical Center,Ethiopian Restaurant,Empanada Restaurant,Event Space
3,M1G,Scarborough,Woburn,43.770992,-79.216917,1.0,Coffee Shop,Korean Restaurant,Women's Store,Eastern European Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Electronics Store
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,1.0,Hakka Restaurant,Athletics & Sports,Caribbean Restaurant,Thai Restaurant,Bakery,Bank,Fried Chicken Joint,Women's Store,Drugstore,Doner Restaurant


In [55]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
#        color=rainbow[cluster-1],
        fill=True,
#        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [56]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
91,Etobicoke,0.0,Baseball Field,Pool,Women's Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store
97,North York,0.0,Baseball Field,Furniture / Home Store,Women's Store,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Discount Store


In [57]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,1.0,Fast Food Restaurant,Print Shop,Eastern European Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Electronics Store
2,Scarborough,1.0,Pizza Place,Breakfast Spot,Rental Car Location,Intersection,Electronics Store,Mexican Restaurant,Medical Center,Ethiopian Restaurant,Empanada Restaurant,Event Space
3,Scarborough,1.0,Coffee Shop,Korean Restaurant,Women's Store,Eastern European Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Electronics Store
4,Scarborough,1.0,Hakka Restaurant,Athletics & Sports,Caribbean Restaurant,Thai Restaurant,Bakery,Bank,Fried Chicken Joint,Women's Store,Drugstore,Doner Restaurant
5,Scarborough,1.0,Playground,Convenience Store,Eastern European Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Women's Store
6,Scarborough,1.0,Discount Store,Bus Station,Department Store,Chinese Restaurant,Coffee Shop,Women's Store,Dumpling Restaurant,Doner Restaurant,Donut Shop,Drugstore
7,Scarborough,1.0,Bakery,Bus Line,Soccer Field,Metro Station,Bus Station,Intersection,Fast Food Restaurant,Park,Empanada Restaurant,Ethiopian Restaurant
8,Scarborough,1.0,Movie Theater,Motel,American Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Women's Store
9,Scarborough,1.0,College Stadium,Skating Rink,General Entertainment,Café,Ethiopian Restaurant,Empanada Restaurant,Event Space,Electronics Store,Eastern European Restaurant,Dim Sum Restaurant
10,Scarborough,1.0,Indian Restaurant,Pet Store,Vietnamese Restaurant,Latin American Restaurant,Chinese Restaurant,Women's Store,Dumpling Restaurant,Dog Run,Doner Restaurant,Donut Shop


In [58]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
48,Central Toronto,2.0,Tennis Court,Park,Eastern European Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Electronics Store


In [60]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Scarborough,3.0,Bar,Women's Store,Electronics Store,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Empanada Restaurant,Discount Store


In [61]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,Scarborough,4.0,Playground,Park,Eastern European Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Electronics Store
23,North York,4.0,Convenience Store,Park,Bank,Bar,Women's Store,Electronics Store,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant
25,North York,4.0,Food & Drink Shop,Park,Electronics Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Women's Store
30,North York,4.0,Airport,Park,Women's Store,Electronics Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant
40,East York,4.0,Convenience Store,Coffee Shop,Park,Women's Store,Eastern European Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
44,Central Toronto,4.0,Bus Line,Park,Swim School,Women's Store,Dumpling Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant
50,Downtown Toronto,4.0,Park,Playground,Building,Trail,Drugstore,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant
74,York,4.0,Park,Women's Store,Market,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Diner
79,North York,4.0,Basketball Court,Construction & Landscaping,Bakery,Park,Empanada Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store
90,Etobicoke,4.0,Pool,River,Park,Women's Store,Drugstore,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop
